# Data-sources exploration using `eo-learn`

This notebook shows some examples on how to retrieve EO and non-EO data using `eo-learn`. 

The steps are as follow:
 * split area of interest into easy-to-process EOPatches
 * add Sentinel-2 imaging data
 * add vector and raster data from OSM
 * add Sentinel-1 imaging data

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Add generic packages

In [ ]:
import os

from matplotlib import dates
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Polygon, box, shape, mapping
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import overpass

Set path to data

In [ ]:
from pathlib import Path
data_dir = Path('./../data/')
os.listdir(data_dir)

`eo-learn` and `sentinelhub` imports

In [ ]:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, Dependency, FeatureType, CompositeTask
from eolearn.io import S2L1CWCSInput, L8L1CWCSInput, DEMWCSInput, AddGeopediaFeature, ExportToTiff
from eolearn.io.sentinelhub_service import S1IWWCSInput
from eolearn.geometry import VectorToRaster

In [ ]:
from sentinelhub import BBoxSplitter, BBox, CRS, DataSource, transform_bbox, GeopediaFeatureIterator

## 1. Split country into smaller bounding boxes <a id='splitter'></a>

Load shapefile of Denmark

In [ ]:
country_filename = data_dir/'denmark.geojson'
country = gpd.read_file(str(country_filename))
country.plot()
country.crs

Set CRS to UTM

In [ ]:
country_crs = CRS.UTM_32N
country = country.to_crs(crs={'init':CRS.ogc_string(country_crs)})
country.plot()
country.crs

Get size of country in pixels to decide number and size of bounding boxes

In [ ]:
country_shape = country.geometry.values.tolist()[-1]
width_pix = int((country_shape.bounds[2]-country_shape.bounds[0])/10)
height_pix = int((country_shape.bounds[3]-country_shape.bounds[1])/10)
print('Dimension of the area is {} x {} pixels'.format(width_pix, height_pix))

Split area into 45x35 boxes bounding 

In [ ]:
bbox_splitter = BBoxSplitter([country_shape], country_crs, (45, 35))

In [ ]:
geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_splitter.bbox_list]
idxs_x = [info['index_x'] for info in bbox_splitter.info_list]
idxs_y = [info['index_y'] for info in bbox_splitter.info_list]

df = pd.DataFrame({'index_x':idxs_x, 'index_y':idxs_y})
gdf = gpd.GeoDataFrame(df, crs={'init':CRS.ogc_string(bbox_splitter.bbox_list[0].crs)}, geometry=geometry)

In [ ]:
gdf.head()

Plot results

In [ ]:
fontdict = {'family': 'monospace', 'weight': 'normal', 'size': 14}
# if bboxes have all same size, estimate offset
xl, yl, xu, yu = gdf.geometry[0].bounds
xoff, yoff = (xu-xl)/3, (yu-yl)/5
# figure
fig, ax = plt.subplots(figsize=(45,35))
gdf.plot(ax=ax, facecolor='w', edgecolor='r', alpha=0.5, linewidth=2)
country.plot(ax=ax, facecolor='w', edgecolor='b', alpha=0.5, linewidth=2.5)
ax.set_title('Denmark tiled in a 45 x 35 grid');
# add annotiation text
for idx in gdf.index:
    eop_name = '{0}x{1}'.format(gdf.index_x[idx], gdf.index_y[idx])
    centroid, = list(gdf.geometry[idx].centroid.coords)
    ax.text(centroid[0]-xoff, centroid[1]+yoff, '{}'.format(idx), fontdict=fontdict)
    ax.text(centroid[0]-xoff, centroid[1]-yoff, eop_name, fontdict=fontdict)

## 2. Retrieve S2 L1C data <a id="sentinel-2"></a>

In [ ]:
s2_l1c_rgb = S2L1CWCSInput('TRUE-COLOR-S2-L1C', resx='10m', resy='10m', maxcc=0.1)
s2_l1c_ndvi = S2L1CWCSInput('NDVI', resx='10m', resy='10m', maxcc=0.1)

In [ ]:
time_interval = ['2019-05-01','2019-09-01']
idx = ???
bbox = bbox_splitter.bbox_list[idx]

Download TRUE-COLOR

In [ ]:
eop_s2 = s2_l1c_rgb.execute(bbox=bbox, time_interval=time_interval)

In [ ]:
eop_s2

Download NDVI

In [ ]:
eop_s2 = s2_l1c_ndvi.execute(eop_s2)

In [ ]:
eop_s2

In [ ]:
eop_s2.timestamp

Plot RGB of time frames

In [ ]:
time_idx = 0
fig, ax = plt.subplots(figsize=(15,15))
im = ax.imshow(1.5*eop_s2.data['TRUE-COLOR-S2-L1C'][time_idx])

Plot the median RGB values

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
im = ax.imshow(1.5*np.median(eop_s2.data['TRUE-COLOR-S2-L1C'], axis=0).squeeze())

Plot the median NDVI values

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
im = ax.imshow(np.median(eop_s2.data['NDVI'], axis=0).squeeze(), cmap=plt.cm.YlGn)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

Plot temporal NDVI of a given location

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.plot(dates.date2num(eop_s2.timestamp), eop_s2.data['NDVI'][:, 100, 550, :].squeeze(), 'g')
ax.set_title('NDVI evolution')
ax.set_xticks(dates.date2num(eop_s2.timestamp));
ax.set_xticklabels([timestamp.date().isoformat() for timestamp in eop_s2.timestamp], rotation=45, ha='right');
ax.set_ylabel('NDVI');

## 3. Add information from OSM <a id="osm"></a>

This task is under-review and will soon make it into the released version

In [ ]:
class OSMInput(EOTask):
    """ Use OpenStreetMap (OSM) data from an Overpass API as input to a VECTOR_TIMELESS feature.
    In case of timeouts or too many requests against the main Overpass endpoint, find additional
    endpoints at see other options https://wiki.openstreetmap.org/wiki/Overpass_API#Public_Overpass_API_instances
    :param feature_name: EOPatch feature into which data will be imported
    :type feature_name: (FeatureType, str)
    :param query: Overpass API Querystring: https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL
    :type query: str
    :param polygonize: Whether or not to treat ways as polygons, defaults to True
    :type polygonize: bool
    :param overpass_opts: Options to pass to the Overpass API constructor, see: https://github.com/mvexel/overpass-api-python-wrapper#api-constructor
    :type overpass_opts: dict
    """

    def __init__(self, feature_name, query, polygonize=True, overpass_opts={}):
        self.feature_name = feature_name
        self.query = query
        self.polygonize = polygonize
        self.api = overpass.API(overpass_opts)

    def execute(self, eopatch):
        """ Execute function which adds new VECTOR_TIMELESS layer to the EOPatch
        :param eopatch: input EOPatch
        :type eopatch: EOPatch
        :return: New EOPatch with added VECTOR_TIMELESS layer
        :rtype: EOPatch
        """

        if self.feature_name is None:
            raise ValueError('\'feature_name\' is a required parameter.')
        if self.query is None:
            raise ValueError('Please provide a \'query\', https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL')
        if not eopatch.bbox:
            raise ValueError('Each EOPatch requires a bbox to fetch data')

        # handling for various bounds variables
        ll_bounds = eopatch.bbox.transform(CRS.WGS84)
        clip_shape = box(*ll_bounds)
        osm_bbox = tuple([*ll_bounds.reverse()])

        # make the overpass request
        response = self.api.get(f'{self.query}{osm_bbox}', verbosity='geom')

        # clip geometries to bounding box
        for feat in response['features']:
            geom = Polygon(shape(feat['geometry']))
            if self.polygonize:
                geom = geom.convex_hull
            clipped_geom = geom.intersection(clip_shape)
            feat['geometry'] = mapping(clipped_geom)


        # import to geopandas, transform and return
        gdf = gpd.GeoDataFrame.from_features(response['features'])
        gdf.crs = {'init' :'epsg:4326'}
        gdf = gdf.to_crs({'init': eopatch.bbox.crs.ogc_string()})
        eopatch[FeatureType.VECTOR_TIMELESS][self.feature_name] = gdf
        return eopatch

In [ ]:
osm_task = OSMInput('residential', 'way["landuse"="residential"]', polygonize=False)

In [ ]:
osm_task.execute(eop_s2)

We now burn the vector feature into a raster mask. 

The same task can be used to burn to raster any vector data stored in a shapefile.

In [ ]:
# help(VectorToRaster)

In [ ]:
rasterise = VectorToRaster((FeatureType.VECTOR_TIMELESS, 'residential'), 
                           (FeatureType.MASK_TIMELESS, 'RESIDENTIAL_MASK'), 
                           values=1, raster_shape=(1007, 1002))
rasterise.execute(eop_s2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(1.5*np.median(eop_s2.data['TRUE-COLOR-S2-L1C'], axis=0).squeeze())
ax.imshow(eop_s2.mask_timeless['RESIDENTIAL_MASK'].squeeze(), alpha=.3)

Tasks can be created to retrieve vector data from Geopedia, or from other geospatial databases.

## 4. Retrieve S1 data <a id="sentinel-1"></a>

In [ ]:
s1_iw_des = S1IWWCSInput('IW_VV', resx='10m', resy='10m', orbit='descending')
s1_iw_asc = S1IWWCSInput('IW_VV', resx='10m', resy='10m', orbit='ascending')

In [ ]:
eop_s1_iw_des = s1_iw_des.execute(bbox=bbox, time_interval=['2019-07-01','2019-08-01'])

In [ ]:
eop_s1_iw_des

[VV-polarised Timescan Composite](https://github.com/ESA-PhiLab/OpenSarToolkit/blob/master/README.md)

In [ ]:
vv_des_r = np.percentile(eop_s1_iw_des.data['IW_VV'], 80, axis=0)[..., [0]]
vv_des_g = np.percentile(eop_s1_iw_des.data['IW_VV'], 20, axis=0)[..., [0]]
vv_des_b = np.std(eop_s1_iw_des.data['IW_VV'], axis=0)[..., [0]]

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(np.concatenate((vv_des_r, vv_des_r, vv_des_b), axis=-1))

In [ ]:
eop_s1_iw_asc = s1_iw_asc.execute(bbox=bbox, time_interval=['2019-07-01','2019-08-01'])

In [ ]:
eop_s1_iw_asc

In [ ]:
vv_asc_r = np.percentile(eop_s1_iw_asc.data['IW_VV'], 80, axis=0)[..., [0]]
vv_asc_g = np.percentile(eop_s1_iw_asc.data['IW_VV'], 20, axis=0)[..., [0]]
vv_asc_b = np.std(eop_s1_iw_asc.data['IW_VV'], axis=0)[..., [0]]

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(np.concatenate((vv_asc_r, vv_asc_r, vv_asc_b), axis=-1))

Similarly, Sentinel-2 L2A data can be added, as well as Digital Elevation data